# preparar datasets

In [18]:
import pandas as pd

### Promotores

In [19]:
df_promoter = pd.read_csv("../data/PromoterSet.tsv", sep="\t")
print(df_promoter.shape)
print(df_promoter.columns.tolist())

(4052, 12)
['pmId', 'pmName', 'strand', 'posTSS', 'sigmaFactor', 'pmSequence', 'firstGeneName', 'distToFirstGene', 'pmEvidence', 'addEvidence', 'confidenceLevel', 'pmids(reference)']


In [20]:
df_promoter.head(2)

,pmId,pmName,strand,posTSS,sigmaFactor,pmSequence,firstGeneName,distToFirstGene,pmEvidence,addEvidence,confidenceLevel,pmids(reference)
0,RDBECOLIPMC00001,spyp,reverse,1825688.0,sigma70,acactttcattgttttaccgttgctctgattaattgacgctaaagt...,spy,-63.0,[COMP-AINF:W][COMP-HINF-POSITIONAL-IDENTIFICAT...,NaN,STRONG,14529615;16861804;9068658
1,RDBECOLIPMC00002,yfeKp,forward,2537315.0,sigma24,ccgatgatcctcatcgtaatccaaccgaaactttacctgattctgg...,yfeK,-27.0,[COMP-AINF:W][COMP-AINF-POSITIONAL-IDENTIFICAT...,NaN,STRONG,14529615;16336047


In [36]:
df_promoter["confidenceLevel"] = df_promoter["confidenceLevel"].replace({"C": "CONFIRMED", "S": "STRONG", "W": "WEAK"})

In [38]:
df_promoter["confidenceLevel"].value_counts()

confidenceLevel
WEAK         2517
STRONG       1455
CONFIRMED      79
Name: count, dtype: int64

In [40]:
df_promoter.to_csv("../data/PromoterSet.tsv", sep="\t", index=False)

### transcription_units

In [42]:
df_tu = pd.read_csv("../data/TUSet.tsv", sep="\t")
print(df_tu.shape)
print(df_tu.columns.tolist())

(3742, 9)
['tuId', 'tuName', 'operonName', 'tuGenes', 'pmName', 'tuEvidence', 'addEvidence', 'confidenceLevel', 'pmids']


In [43]:
df_tu["confidenceLevel"] = df_tu["confidenceLevel"].replace({"C": "CONFIRMED", "S": "STRONG", "W": "WEAK"})

In [46]:
df_tu["pmids"].value_counts()

Series([], Name: count, dtype: int64)

In [47]:
df_tu.to_csv("../data/TUSet.tsv", sep="\t", index=False)

### Evidence

In [12]:
df_evidence = pd.read_csv("../data/EvidenceSet.tsv", sep="\t")
df_evidence.head()

,evidence_code,evidence_name,evidence_group,confidence_level,object_type,evidence_category
0,EXP-IGI-FUNC-COMPLEMENTATION,Inferred by functional complementation,NaN,STRONG,Transcription Factors,Classical experiment
1,EXP-IDA-UNPURIFIED-PROTEIN-NH,Assay of unpurified protein expressed in its n...,NaN,WEAK,Transcription Factors,Classical experiment
2,EXP-CHIP-EXO-MANUAL,ChIP-exo evidence used in manual assertion,4.0,WEAK,Regulatory Interactions,High-throughput binding
3,EXP-IEP-GENE-EXPRESSION-ANALYSIS,Gene expression analysis,8.0,WEAK,Regulatory Interactions;Transcription Factors,Classical expression
4,EXP-HTP-HEP-MICROARRAY,High-throughput expression microarray evidence,8.0,WEAK,Transcription Units;Regulatory Interactions,High-throughput expression


### Regulator Interaction

In [51]:
df_ri = pd.read_csv("../data/RISet.tsv", sep="\t")
df_ri.head()

,riId,riType,regulatorId(TranscriptionFactor),regulatorName,regulatorActiveConformation(CNF)Name,regulatorRegulatorySite(tfrs)ID,tfrsLeftEndPosition,tfrsRightEndPosition,strand,tfrsSequence,...,relativeDistanceFromCenterPositionOfTFRStoStartOfFirstGene,transcriptionUnitOrGeneRegulatedByTF,confidenceLevel,tfrsEvidence,riEvidence,additiveEvidence,riEvidenceTech,riEvidenceCategory,tfrsPMIDS,riPMIDS
0,RDBECOLIRIC00001,tf-promoter,RDBECOLITFC00023,GadW,GadW,RDBECOLIBSC01689,3653866.0,3653885.0,forward,atcagacaggTTTACGTTACTATCAGGCATatcacctcag,...,-85.5,RDBECOLITUC03133:slp-dctR,WEAK,COMP-AINF-SIMILAR-TO-CONSENSUS:W,EXP-IEP-GENE-EXPRESSION-ANALYSIS:W,NaN,expression|non-experimental,Classical expression|non-experimental,18808381,12730179;18808381
1,RDBECOLIRIC00002,tf-promoter,RDBECOLITFC00023,GadW,GadW,RDBECOLIBSC01690,3656835.0,3656854.0,reverse,tgaaataaaaATATCTGATTTTGATATTTTccatcaacat,...,-104.5,RDBECOLITUC03315:hdeAB-yhiD,STRONG,COMP-HINF-SIMILAR-TO-CONSENSUS:W;COMP-AINF-SIM...,EXP-IEP-GENE-EXPRESSION-ANALYSIS:W,NaN,expression|non-experimental|binding in vitro,Classical expression|non-experimental|Classica...,18083817;18808381,12730179;18083817;18808381
2,RDBECOLIRIC00003,tf-promoter,RDBECOLITFC00088,GalR,GalR-D-galactose,RDBECOLIBSC01691,3088004.0,3088019.0,forward,atcatcacaaCTGAAACCGATTACACcaaccacaac,...,-272.5,RDBECOLITUC02515:galP,CONFIRMED,COMP-AINF-SIMILAR-TO-CONSENSUS:W;EXP-IDA-BINDI...,NaN,AE(EXP-IMP-SITE-MUTATION/EXP-IDA-BINDING-OF-PU...,non-experimental|binding in vitro|binding in v...,non-experimental|Classical binding,19007420,19007420
3,RDBECOLIRIC00004,tf-promoter,RDBECOLITFC00033,GalS,GalS-D-galactose,RDBECOLIBSC01692,3088004.0,3088019.0,forward,atcatcacaaCTGAAACCGATTACACcaaccacaac,...,-272.5,RDBECOLITUC02515:galP,CONFIRMED,COMP-AINF-SIMILAR-TO-CONSENSUS:W;EXP-IDA-BINDI...,NaN,AE(EXP-IMP-SITE-MUTATION/EXP-IDA-BINDING-OF-PU...,non-experimental|binding in vitro|binding in v...,non-experimental|Classical binding,19007420,19007420
4,RDBECOLIRIC00005,tf-promoter,RDBECOLITFC00088,GalR,GalR-D-galactose,RDBECOLIBSC01693,3088069.0,3088084.0,forward,ttattcggttTTCACAGTTGTTACATttcttttcag,...,-207.5,RDBECOLITUC02515:galP,CONFIRMED,COMP-AINF-SIMILAR-TO-CONSENSUS:W;EXP-IDA-BINDI...,NaN,AE(EXP-IMP-SITE-MUTATION/EXP-IDA-BINDING-OF-PU...,non-experimental|binding in vitro|binding in v...,non-experimental|Classical binding,19007420,19007420


In [15]:
df_ri["confidenceLevel"] = df_ri["confidenceLevel"].replace({"C": "CONFIRMED", "S": "STRONG", "W": "WEAK"})

In [19]:
df_ri.to_csv("../data/RISet.tsv", sep="\t", index=False)

### Terminator

In [25]:
df_terminator = pd.read_csv("../data/TerminatorSet.tsv", sep="\t")
df_terminator.head()

,terminatorId,leftEndPos,rightEndPos,strand,sequence,relatedTus,type,operonName,pmids,evidences,confidenceLevel
0,RDBECOLITMC00001,3407286,3407302,forward,agcgtcaaaaGGCCGGATTTTCCGGCCttttttatta,accBC:accBp2;accBC:accBp1,rho-independent,accBC,8226664,NaN,None
1,RDBECOLITMC00002,3418143,3418177,forward,gataaatcagAAACATAAAGGCGCTTTCGGGTGCCTTTATTATTTc...,acrEF:acrEp,rho-independent,acrEF,1720861,NaN,None
2,RDBECOLITMC00003,4639539,4639572,reverse,taccaccgtcAAAAAAAACGGCGCTTTTTAGCGCCGTTTTTATTtt...,arcA:arcAp1;arcA:arcAp2;arcA:arcAp3;arcA:arcAp...,rho-independent,arcA,2984198,NaN,None
3,RDBECOLITMC00004,2738912,2738940,forward,ctgatgaaaaGGTGCCGGATGATGTGAATCATCCGGCACtggattatta,pheLA:pheLp;aroF-tyrA:aroFp,rho-independent,pheLA;aroF-tyrA,6396419,NaN,None
4,RDBECOLITMC00005,786795,786821,forward,tagcaacaaaAAAGCCGACTCACTTGCAGTCGGCTTTctcattttaa,aroG:aroGp,rho-independent,aroG,"1970563, 6125934",NaN,None


In [28]:
df_terminator["confidenceLevel"].unique()

array(['None ', nan], dtype=object)

In [29]:
df_terminator.pop("confidenceLevel")

0      None 
1      None 
2      None 
3      None 
4      None 
       ...  
366    None 
367    None 
368    None 
369    None 
370      NaN
Name: confidenceLevel, Length: 371, dtype: object

In [31]:
df_terminator["evidences"].unique()

array([nan])

In [32]:
df_terminator.pop("evidences")

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
366   NaN
367   NaN
368   NaN
369   NaN
370   NaN
Name: evidences, Length: 371, dtype: float64

In [33]:
df_terminator.to_csv("../data/TerminatorSet.tsv", sep="\t", index=False)

### GeneProductSet

In [6]:
df_gene_product = pd.read_csv("../data/GeneProductSet.tsv", sep="\t")
df_gene_product.head(2)

,geneId,geneName,bnumber,leftEndPosition,rightEndPosition,strand,productName,geneEvidences,genePMIDS,confidenceLevel,relatedBnumbers,otherDbsIds
0,RDBECOLIGNC00001,alr,b4053,4265782.0,4266861.0,forward,alanine racemase 1,NaN,NaN,NaN,NaN,[STRING:511145.b4053][ASAP:ABE-0013272][ECHOBA...
1,RDBECOLIGNC00002,modB,b0764,795862.0,796551.0,forward,molybdate ABC transporter membrane subunit,NaN,NaN,NaN,NaN,[STRING:511145.b0764][ASAP:ABE-0002603][ECHOBA...


In [7]:
df_gene_product["geneEvidences"].unique()

array([nan])

In [9]:
df_gene_product["confidenceLevel"].unique()

array([nan])

### Operon

In [15]:
df_operon = pd.read_csv("../data/OperonSet.tsv", sep="\t")
df_operon.head(2)

,operonId,operonName,firstGeneLeftPosition,lastGeneRightPosition,strand,numberOfGenes,operonGenes,operonEvidence,confidenceLevel
0,RDBECOLIOPC00001,ispDF,2871301,2872490,reverse,2,ispD;ispF,[COMP-HINF:W][IC-ADJ-GENES-SAME-BIO-PROCESS:W],W
1,RDBECOLIOPC00002,serX,1097565,1097652,reverse,1,serX,[EXP-IDA-BOUNDARIES-DEFINED:W],W


In [20]:
df_operon["confidenceLevel"].unique()

array(['W ', ' ', 'S ', 'WEAK'], dtype=object)

In [21]:
df_operon["confidenceLevel"] = df_operon["confidenceLevel"].replace({"C": "CONFIRMED", "S": "STRONG", "W": "WEAK", "W ": "WEAK","S ": "STRONG", " ": ""})

In [22]:
df_operon.head(2)

,operonId,operonName,firstGeneLeftPosition,lastGeneRightPosition,strand,numberOfGenes,operonGenes,operonEvidence,confidenceLevel
0,RDBECOLIOPC00001,ispDF,2871301,2872490,reverse,2,ispD;ispF,[COMP-HINF:W][IC-ADJ-GENES-SAME-BIO-PROCESS:W],WEAK
1,RDBECOLIOPC00002,serX,1097565,1097652,reverse,1,serX,[EXP-IDA-BOUNDARIES-DEFINED:W],WEAK


In [23]:
df_operon.to_csv("../data/OperonSet.tsv", sep="\t", index=False)

In [24]:
df_operon = pd.read_csv("../data/OperonSet.tsv", sep="\t")
df_operon.head(2)

,operonId,operonName,firstGeneLeftPosition,lastGeneRightPosition,strand,numberOfGenes,operonGenes,operonEvidence,confidenceLevel
0,RDBECOLIOPC00001,ispDF,2871301,2872490,reverse,2,ispD;ispF,[COMP-HINF:W][IC-ADJ-GENES-SAME-BIO-PROCESS:W],WEAK
1,RDBECOLIOPC00002,serX,1097565,1097652,reverse,1,serX,[EXP-IDA-BOUNDARIES-DEFINED:W],WEAK


### Regulator

In [25]:
df_regulator = pd.read_csv("../data/RegulatorSet.tsv", sep="\t")
df_regulator.head(2)

,regulatorId,regulatorName,regulatorSynonyms,regulatorType,geneCodingForRegulator,geneBnumberCodingForRegulator,regulatorActiveConformations,regulatorInactiveConformations,regulatorActiveConformationsSynonyms,regulatorInactiveConformationsSynonyms,regulatorActiveConformationsEffector,regulatorInactiveConformationsEffector,regulatorActiveConformationsEffectorSynonyms,regulatorInactiveConformationsEffectorSynonyms,symmetry,regulatorEvidences,additiveEvidences,confidenceLevel,regulatorConformationPMID
0,RDBECOLITFC00001,ExuR,"ExuR;negative regulator of exu regulon, exuT, ...",transcriptionFactor,exuR,b3094,"ExuR;ExuR-&alpha;-D-galacturonate, &alpha;-D-g...",ExuR-&alpha;-D-glucuronate,"ExuR;negative regulator of exu regulon, exuT, ...",NaN,&alpha;-D-galacturonate,&alpha;-D-glucuronate,&alpha;-D-galacturonic acid,&alpha;-D-glucuronic acid;&alpha;-D-glucopyran...,|INVERTED-REPEAT|,[EXP:W],NaN,W,6357945
1,RDBECOLITFC00002,AsnC,AsnC,transcriptionFactor,asnC,b3743,DNA-binding transcriptional dual regulator AsnC,AsnC-L-asparagine,AsnC,NaN,NaN,L-asparagine,NaN,asparagine;&alpha;-aminosuccinamic acid;(-)-as...,|INVERTED-REPEAT|,[COMP-HINF-FN-FROM-SEQ:W][EXP-IEP:W][EXP-IPI:W],NaN,W,6357950; 7686882; 16528101; 2864330


In [26]:
df_regulator["confidenceLevel"].unique()

array(['W', 'S', nan], dtype=object)

In [27]:
df_regulator["confidenceLevel"] = df_regulator["confidenceLevel"].replace({"C": "CONFIRMED", "S": "STRONG", "W": "WEAK"})

In [28]:
df_regulator.to_csv("../data/RegulatorSet.tsv", sep="\t", index=False)

In [29]:
df_regulator = pd.read_csv("../data/RegulatorSet.tsv", sep="\t")
df_regulator.head(2)

,regulatorId,regulatorName,regulatorSynonyms,regulatorType,geneCodingForRegulator,geneBnumberCodingForRegulator,regulatorActiveConformations,regulatorInactiveConformations,regulatorActiveConformationsSynonyms,regulatorInactiveConformationsSynonyms,regulatorActiveConformationsEffector,regulatorInactiveConformationsEffector,regulatorActiveConformationsEffectorSynonyms,regulatorInactiveConformationsEffectorSynonyms,symmetry,regulatorEvidences,additiveEvidences,confidenceLevel,regulatorConformationPMID
0,RDBECOLITFC00001,ExuR,"ExuR;negative regulator of exu regulon, exuT, ...",transcriptionFactor,exuR,b3094,"ExuR;ExuR-&alpha;-D-galacturonate, &alpha;-D-g...",ExuR-&alpha;-D-glucuronate,"ExuR;negative regulator of exu regulon, exuT, ...",NaN,&alpha;-D-galacturonate,&alpha;-D-glucuronate,&alpha;-D-galacturonic acid,&alpha;-D-glucuronic acid;&alpha;-D-glucopyran...,|INVERTED-REPEAT|,[EXP:W],NaN,WEAK,6357945
1,RDBECOLITFC00002,AsnC,AsnC,transcriptionFactor,asnC,b3743,DNA-binding transcriptional dual regulator AsnC,AsnC-L-asparagine,AsnC,NaN,NaN,L-asparagine,NaN,asparagine;&alpha;-aminosuccinamic acid;(-)-as...,|INVERTED-REPEAT|,[COMP-HINF-FN-FROM-SEQ:W][EXP-IEP:W][EXP-IPI:W],NaN,WEAK,6357950; 7686882; 16528101; 2864330


### Trancription Factor

In [31]:
df_tf = pd.read_csv("../data/TFSet.tsv", sep="\t")
df_tf.head(2)

,tfId,tfName,tfSynonyms,geneCodingForTF,geneBnumberCodingForTF,tfActiveConformations,tfInactiveConformations,tfActiveConformationsSynonyms,tfInactiveConformationsSynonyms,tfActiveConformationsEffector,tfInactiveConformationsEffector,tfActiveConformationsEffectorSynonyms,tfInactiveConformationsEffectorSynonyms,symmetry,tfEvidences,additiveEvidences,confidenceLevel,tfConformationPMID
0,RDBECOLITFC00001,ExuR,"ExuR;negative regulator of exu regulon, exuT, ...",exuR,b3094,"ExuR;ExuR-&alpha;-D-galacturonate, &alpha;-D-g...",ExuR-&alpha;-D-glucuronate,"ExuR;negative regulator of exu regulon, exuT, ...",NaN,&alpha;-D-galacturonate,&alpha;-D-glucuronate,&alpha;-D-galacturonic acid,&alpha;-D-glucuronic acid;&alpha;-D-glucopyran...,|INVERTED-REPEAT|,[EXP:W],NaN,W,6357945
1,RDBECOLITFC00002,AsnC,AsnC,asnC,b3743,DNA-binding transcriptional dual regulator AsnC,AsnC-L-asparagine,AsnC,NaN,NaN,L-asparagine,NaN,asparagine;&alpha;-aminosuccinamic acid;(-)-as...,|INVERTED-REPEAT|,[COMP-HINF-FN-FROM-SEQ:W][EXP-IEP:W][EXP-IPI:W],NaN,W,6357950; 7686882; 16528101; 2864330


In [32]:
df_tf["confidenceLevel"].unique()

array(['W', 'S', nan], dtype=object)

In [33]:
df_tf["confidenceLevel"] = df_tf["confidenceLevel"].replace({"C": "CONFIRMED", "S": "STRONG", "W": "WEAK"})

In [34]:
df_tf.to_csv("../data/TFSet.tsv", sep="\t", index=False)

In [35]:
df_tf = pd.read_csv("../data/TFSet.tsv", sep="\t")
df_tf.head(2)

,tfId,tfName,tfSynonyms,geneCodingForTF,geneBnumberCodingForTF,tfActiveConformations,tfInactiveConformations,tfActiveConformationsSynonyms,tfInactiveConformationsSynonyms,tfActiveConformationsEffector,tfInactiveConformationsEffector,tfActiveConformationsEffectorSynonyms,tfInactiveConformationsEffectorSynonyms,symmetry,tfEvidences,additiveEvidences,confidenceLevel,tfConformationPMID
0,RDBECOLITFC00001,ExuR,"ExuR;negative regulator of exu regulon, exuT, ...",exuR,b3094,"ExuR;ExuR-&alpha;-D-galacturonate, &alpha;-D-g...",ExuR-&alpha;-D-glucuronate,"ExuR;negative regulator of exu regulon, exuT, ...",NaN,&alpha;-D-galacturonate,&alpha;-D-glucuronate,&alpha;-D-galacturonic acid,&alpha;-D-glucuronic acid;&alpha;-D-glucopyran...,|INVERTED-REPEAT|,[EXP:W],NaN,WEAK,6357945
1,RDBECOLITFC00002,AsnC,AsnC,asnC,b3743,DNA-binding transcriptional dual regulator AsnC,AsnC-L-asparagine,AsnC,NaN,NaN,L-asparagine,NaN,asparagine;&alpha;-aminosuccinamic acid;(-)-as...,|INVERTED-REPEAT|,[COMP-HINF-FN-FROM-SEQ:W][EXP-IEP:W][EXP-IPI:W],NaN,WEAK,6357950; 7686882; 16528101; 2864330


# crear create_sql_agent

In [1]:
from langchain_community.utilities import SQLDatabase
from langchain_experimental.tools import PythonAstREPLTool
from sqlalchemy import create_engine

engine = create_engine("sqlite:///../regulon.db")

In [55]:
df_promoter.to_sql("promotores", engine, index=False, if_exists="replace")
df_tu.to_sql("transcription_units", engine, index=False, if_exists="replace")
df_ri.to_sql("regulatory_interactions", engine, index=False, if_exists="replace")
df_terminator.to_sql("terminators", engine, index=False, if_exists="replace")
df_evidence.to_sql("evidences", engine, index=False, if_exists="replace")

82

In [37]:
df_gene_product.to_sql("gene_products", engine, index=False, if_exists="replace")
df_operon.to_sql("operons", engine, index=False, if_exists="replace")
df_regulator.to_sql("regulators", engine, index=False, if_exists="replace")
df_tf.to_sql("transcription_factors", engine, index=False, if_exists="replace")

240

In [8]:
db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())

sqlite
['evidences', 'gene_products', 'operons', 'promotores', 'regulators', 'regulatory_interactions', 'terminators', 'transcription_factors', 'transcription_units']


In [39]:
print(db.run("SELECT name FROM sqlite_master WHERE type='table';"))

[('promotores',), ('transcription_units',), ('regulatory_interactions',), ('terminators',), ('evidences',), ('gene_products',), ('operons',), ('regulators',), ('transcription_factors',)]


In [6]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [9]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [6]:
agent_executor.invoke({"input": "Devuelve todos los nombres de los promotores regulados por el sigma factor sigma54"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


evidences, gene_products, operons, promotores, regulators, regulatory_interactions, terminators, transcription_factors, transcription_units
Invoking: `sql_db_schema` with `{'table_names': 'promotores'}`



CREATE TABLE promotores (
	"pmId" TEXT, 
	"pmName" TEXT, 
	strand TEXT, 
	"posTSS" FLOAT, 
	"sigmaFactor" TEXT, 
	"pmSequence" TEXT, 
	"firstGeneName" TEXT, 
	"distToFirstGene" FLOAT, 
	"pmEvidence" TEXT, 
	"addEvidence" TEXT, 
	"confidenceLevel" TEXT, 
	pmids TEXT
)

/*
3 rows from promotores table:
pmId	pmName	strand	posTSS	sigmaFactor	pmSequence	firstGeneName	distToFirstGene	pmEvidence	addEvidence	confidenceLevel	pmids
RDBECOLIPMC00001	spyp	reverse	1825688.0	sigma70	acactttcattgttttaccgttgctctgattaattgacgctaaagtcagtaaagttaatcTcgtcaacacggcacgctact	spy	-63.0	[COMP-AINF:W][COMP-HINF-POSITIONAL-IDENTIFICATION:W][EXP-IDA-TRANSCRIPTION-INIT-MAPPING:S]	None	STRONG	14529615;16861804;9068658 
RDBECOLIP

{'input': 'Devuelve todos los nombres de los promotores regulados por el sigma factor sigma54',
 'output': 'Los nombres de los promotores regulados por el factor sigma "sigma54" son los siguientes:\n\n1. yahEp\n2. yaaUp\n3. crlp2\n4. nikAp2\n5. emrDp\n6. yhjCp\n7. ygfKp\n8. radDp\n9. htpGp3\n10. flhDp2'}

In [7]:
agent_executor.invoke({"input": "son todos?"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


evidences, gene_products, operons, promotores, regulators, regulatory_interactions, terminators, transcription_factors, transcription_units
Invoking: `sql_db_schema` with `{'table_names': 'evidences,gene_products,operons,promotores,regulators,regulatory_interactions,terminators,transcription_factors,transcription_units'}`



CREATE TABLE evidences (
	evidence_code TEXT, 
	evidence_name TEXT, 
	evidence_group FLOAT, 
	confidence_level TEXT, 
	object_type TEXT, 
	"evidence_category " TEXT
)

/*
3 rows from evidences table:
evidence_code	evidence_name	evidence_group	confidence_level	object_type	evidence_category 
EXP-IGI-FUNC-COMPLEMENTATION	Inferred by functional complementation	None	STRONG	Transcription Factors	Classical experiment 
EXP-IDA-UNPURIFIED-PROTEIN-NH	Assay of unpurified protein expressed in its native host	None	WEAK	Transcription Factors	Classical experiment 
EXP-CHIP-EXO-MANUAL	ChIP-exo

{'input': 'son todos?',
 'output': 'I have retrieved the schemas for the tables in the database. Here’s a summary of the tables:\n\n1. **evidences**: Contains information about different types of evidence.\n2. **gene_products**: Details about gene products including gene IDs, names, positions, and confidence levels.\n3. **operons**: Information on operons including their IDs, names, and gene positions.\n4. **promotores**: Contains promoter details including their IDs, sequences, and associated genes.\n5. **regulators**: Information on regulatory factors including their names, types, and associated genes.\n6. **regulatory_interactions**: Details of interactions between regulators and target genes/promoters.\n7. **terminators**: Information on terminators including their sequences and related operons.\n8. **transcription_factors**: Details about transcription factors including their active and inactive conformations.\n9. **transcription_units**: Information on transcription units includi

In [8]:
answer = agent_executor.invoke({"input": "cual es la referencia del promotor yahEp"})
answer



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


evidences, gene_products, operons, promotores, regulators, regulatory_interactions, terminators, transcription_factors, transcription_units
Invoking: `sql_db_schema` with `{'table_names': 'promotores'}`



CREATE TABLE promotores (
	"pmId" TEXT, 
	"pmName" TEXT, 
	strand TEXT, 
	"posTSS" FLOAT, 
	"sigmaFactor" TEXT, 
	"pmSequence" TEXT, 
	"firstGeneName" TEXT, 
	"distToFirstGene" FLOAT, 
	"pmEvidence" TEXT, 
	"addEvidence" TEXT, 
	"confidenceLevel" TEXT, 
	pmids TEXT
)

/*
3 rows from promotores table:
pmId	pmName	strand	posTSS	sigmaFactor	pmSequence	firstGeneName	distToFirstGene	pmEvidence	addEvidence	confidenceLevel	pmids
RDBECOLIPMC00001	spyp	reverse	1825688.0	sigma70	acactttcattgttttaccgttgctctgattaattgacgctaaagtcagtaaagttaatcTcgtcaacacggcacgctact	spy	-63.0	[COMP-AINF:W][COMP-HINF-POSITIONAL-IDENTIFICATION:W][EXP-IDA-TRANSCRIPTION-INIT-MAPPING:S]	None	STRONG	14529615;16861804;9068658 
RDBECOLIP

{'input': 'cual es la referencia del promotor yahEp',
 'output': 'La referencia del promotor "yahEp" es **RDBECOLIPMC00324**.'}

In [60]:
answer = agent_executor.invoke({"input": "cual es la secuencia del promotor que esta relacionado con el id del ri RDBECOLIRIC00001?"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


evidences, promotores, regulatory_interactions, terminators, transcription_units
Invoking: `sql_db_schema` with `{'table_names': 'promotores'}`



CREATE TABLE promotores (
	"pmId" TEXT, 
	"pmName" TEXT, 
	strand TEXT, 
	"posTSS" FLOAT, 
	"sigmaFactor" TEXT, 
	"pmSequence" TEXT, 
	"firstGeneName" TEXT, 
	"distToFirstGene" FLOAT, 
	"pmEvidence" TEXT, 
	"addEvidence" TEXT, 
	"confidenceLevel" TEXT, 
	pmids TEXT
)

/*
3 rows from promotores table:
pmId	pmName	strand	posTSS	sigmaFactor	pmSequence	firstGeneName	distToFirstGene	pmEvidence	addEvidence	confidenceLevel	pmids
RDBECOLIPMC00001	spyp	reverse	1825688.0	sigma70	acactttcattgttttaccgttgctctgattaattgacgctaaagtcagtaaagttaatcTcgtcaacacggcacgctact	spy	-63.0	[COMP-AINF:W][COMP-HINF-POSITIONAL-IDENTIFICATION:W][EXP-IDA-TRANSCRIPTION-INIT-MAPPING:S]	None	STRONG	14529615;16861804;9068658 
RDBECOLIPMC00002	yfeKp	forward	2537315.0	sigma24	ccgatgatcctcatcgtaa

In [61]:
print(answer["output"])

La secuencia del promotor relacionado con el ID del regulador de interacción RDBECOLIRIC00001 es: 

**tatcaggcatatcacctcagaatcagatgaaaactataaagaaatatctattatggttttAatatttgttgataaggatag**.


In [6]:
answer = agent_executor.invoke({"input": "Cuales son las evidencias de la TU araE-ygeA?"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


promotores, transcription_units
Invoking: `sql_db_schema` with `{'table_names': 'promotores'}`



CREATE TABLE promotores (
	"pmId" TEXT, 
	"pmName" TEXT, 
	strand TEXT, 
	"posTSS" FLOAT, 
	"sigmaFactor" TEXT, 
	"pmSequence" TEXT, 
	"firstGeneName" TEXT, 
	"distToFirstGene" FLOAT, 
	"pmEvidence" TEXT, 
	"addEvidence" TEXT, 
	"confidenceLevel" TEXT, 
	"pmids " TEXT
)

/*
3 rows from promotores table:
pmId	pmName	strand	posTSS	sigmaFactor	pmSequence	firstGeneName	distToFirstGene	pmEvidence	addEvidence	confidenceLevel	pmids 
RDBECOLIPMC00001	spyp	reverse	1825688.0	sigma70	acactttcattgttttaccgttgctctgattaattgacgctaaagtcagtaaagttaatcTcgtcaacacggcacgctact	spy	-63.0	[COMP-AINF:W][COMP-HINF-POSITIONAL-IDENTIFICATION:W][EXP-IDA-TRANSCRIPTION-INIT-MAPPING:S]	None	S	14529615;16861804;9068658 
RDBECOLIPMC00002	yfeKp	forward	2537315.0	sigma24	ccgatgatcctcatcgtaatccaaccgaaactttacctgattctggcagtcaaatcggctAtcacaaaa

In [7]:
print(answer["output"])

The evidence for the transcription unit (TU) "araE-ygeA" is as follows:

- **TU ID**: RDBECOLITUC02057
- **TU Name**: araE-ygeA
- **Promoter Name**: araEp
- **Evidence**: [EXP-IDA-TRANSCRIPT-LEN-DETERMINATION:S]

This indicates that the evidence is based on experimental data related to transcript length determination.


# retriever_tool with history

In [23]:
# from langchain import hub
# prompt = hub.pull("langchain-ai/sql-agent-system-prompt")
# prompt.messages

c:\Users\VALENCIA\Documents\cursos\curso-langchain-qa-documents\.venv\Lib\site-packages\langsmith\client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['dialect', 'top_k'], input_types={}, partial_variables={}, template='You are an agent designed to interact with a SQL database.\nGiven an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.\nUnless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.\nYou can order the results by a relevant column to return the most interesting examples in the database.\nNever query for all the columns from a specific table, only ask for the relevant columns given the question.\nYou have access to tools for interacting with the database.\nOnly use the below tools. Only use the information returned by the below tools to construct your final answer.\nYou MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.\n\nDO NOT m

In [1]:
import getpass
import os
from typing import Optional
from dotenv import load_dotenv

load_dotenv()

from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits import create_sql_agent
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import HumanMessage
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

In [25]:
engine = create_engine("sqlite:///../regulon.db")
db = SQLDatabase(engine=engine)
# Crear el modelo y el agente SQL
llm = ChatOpenAI(model="gpt-4o-mini")
agent_sql = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

# Palabras clave para activar el agente SQL
sql_keywords = ["transcription unit", "tu", "tus","transcription units", "promoter", "promotor", 
                "promotores", "sigma factor", "regulon", "evidence","evidencia","ri","regulatory interaction",
                "terminator","terminators","gene","gene product","gene products","operon","operons",
                "regulador","regulator","regulators","tf","transcription factor","transcription factors"]

# Almacén de memoria de conversaciones
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    """Obtiene o crea el historial de conversación para una sesión."""
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Crear un wrapper con memoria
agent_with_chat_history = RunnableWithMessageHistory(
    agent_sql,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [ ]:
# tools = [agent_sql, agent_with_chat_history]
# model_with_tools = llm.bind_tools(tools)

# from langchain_core.messages import HumanMessage
# response = model_with_tools.invoke([HumanMessage(content="Hi!")])

# print(f"ContentString: {response.content}")
# print(f"ToolCalls: {response.tool_calls}")

# from langchain.agents import create_tool_calling_agent

# agent = create_tool_calling_agent(model, tools, prompt)
# from langchain.agents import AgentExecutor

# agent_executor = AgentExecutor(agent=agent, tools=tools)

In [7]:
def process_prompt(prompt: str, session_id: str):
    """Procesa el prompt, decide si usar el agente SQL o solo el LLM, y maneja el historial."""
    chat_history = get_session_history(session_id)
    
    # Decidir si usar el agente SQL o el LLM
    if any(keyword in prompt.lower() for keyword in sql_keywords):
        # Ajustar el límite en las consultas
        if "limit" not in prompt.lower():
            prompt += " sin límite"
        
        # Ejecutar con el agente SQL
        response = agent_with_chat_history.invoke(
            {"input": prompt},
            config={"configurable": {"session_id": session_id}},
        )["output"]
    else:
        # Ejecutar directamente con el modelo, usando el historial
        llm_response = llm.invoke(prompt)
        chat_history.add_message(HumanMessage(content=prompt))
        chat_history.add_message(AIMessage(content=llm_response.content))
        print("llm\n")
        response = llm_response.content

    return response

In [8]:
# Ejemplo de uso
session_id = "123"  # Un identificador único por sesión

In [15]:
prompt_1 = "Devuelve todos los nombres de los promotores regulados por el sigma factor sigma54"
response_1 = process_prompt(prompt_1, session_id)
print(response_1["output"])



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


evidences, gene_products, operons, promotores, regulators, regulatory_interactions, terminators, transcription_factors, transcription_units
Invoking: `sql_db_schema` with `{'table_names': 'promotores'}`



CREATE TABLE promotores (
	"pmId" TEXT, 
	"pmName" TEXT, 
	strand TEXT, 
	"posTSS" FLOAT, 
	"sigmaFactor" TEXT, 
	"pmSequence" TEXT, 
	"firstGeneName" TEXT, 
	"distToFirstGene" FLOAT, 
	"pmEvidence" TEXT, 
	"addEvidence" TEXT, 
	"confidenceLevel" TEXT, 
	pmids TEXT
)

/*
3 rows from promotores table:
pmId	pmName	strand	posTSS	sigmaFactor	pmSequence	firstGeneName	distToFirstGene	pmEvidence	addEvidence	confidenceLevel	pmids
RDBECOLIPMC00001	spyp	reverse	1825688.0	sigma70	acactttcattgttttaccgttgctctgattaattgacgctaaagtcagtaaagttaatcTcgtcaacacggcacgctact	spy	-63.0	[COMP-AINF:W][COMP-HINF-POSITIONAL-IDENTIFICATION:W][EXP-IDA-TRANSCRIPTION-INIT-MAPPING:S]	None	STRONG	14529615;16861804;9068658 
RDBECOLIP

In [9]:
prompt_2 = "¿Cuál es la evidencia del promotor araJ?"
response_2 = process_prompt(prompt_2, session_id)
print(response_2)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


evidences, gene_products, operons, promotores, regulators, regulatory_interactions, terminators, transcription_factors, transcription_units
Invoking: `sql_db_schema` with `{'table_names': 'promotores'}`



CREATE TABLE promotores (
	"pmId" TEXT, 
	"pmName" TEXT, 
	strand TEXT, 
	"posTSS" FLOAT, 
	"sigmaFactor" TEXT, 
	"pmSequence" TEXT, 
	"firstGeneName" TEXT, 
	"distToFirstGene" FLOAT, 
	"pmEvidence" TEXT, 
	"addEvidence" TEXT, 
	"confidenceLevel" TEXT, 
	"pmids(reference)" TEXT
)

/*
3 rows from promotores table:
pmId	pmName	strand	posTSS	sigmaFactor	pmSequence	firstGeneName	distToFirstGene	pmEvidence	addEvidence	confidenceLevel	pmids(reference)
RDBECOLIPMC00001	spyp	reverse	1825688.0	sigma70	acactttcattgttttaccgttgctctgattaattgacgctaaagtcagtaaagttaatcTcgtcaacacggcacgctact	spy	-63.0	[COMP-AINF:W][COMP-HINF-POSITIONAL-IDENTIFICATION:W][EXP-IDA-TRANSCRIPTION-INIT-MAPPING:S]	None	STRONG	14529615;168

In [10]:
chat_history = get_session_history(session_id="123")
chat_history.messages

[HumanMessage(content='¿Cuál es la evidencia del promotor araJ? sin límite', additional_kwargs={}, response_metadata={}),
 AIMessage(content='No se encontraron evidencias relacionadas con el promotor araJ en la base de datos.', additional_kwargs={}, response_metadata={})]

In [9]:
answer = process_prompt("cual es la referencia del promotor yahEp", session_id) 
print(answer)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


evidences, gene_products, operons, promotores, regulators, regulatory_interactions, terminators, transcription_factors, transcription_units
Invoking: `sql_db_schema` with `{'table_names': 'promotores'}`



CREATE TABLE promotores (
	"pmId" TEXT, 
	"pmName" TEXT, 
	strand TEXT, 
	"posTSS" FLOAT, 
	"sigmaFactor" TEXT, 
	"pmSequence" TEXT, 
	"firstGeneName" TEXT, 
	"distToFirstGene" FLOAT, 
	"pmEvidence" TEXT, 
	"addEvidence" TEXT, 
	"confidenceLevel" TEXT, 
	"pmids(reference)" TEXT
)

/*
3 rows from promotores table:
pmId	pmName	strand	posTSS	sigmaFactor	pmSequence	firstGeneName	distToFirstGene	pmEvidence	addEvidence	confidenceLevel	pmids(reference)
RDBECOLIPMC00001	spyp	reverse	1825688.0	sigma70	acactttcattgttttaccgttgctctgattaattgacgctaaagtcagtaaagttaatcTcgtcaacacggcacgctact	spy	-63.0	[COMP-AINF:W][COMP-HINF-POSITIONAL-IDENTIFICATION:W][EXP-IDA-TRANSCRIPTION-INIT-MAPPING:S]	None	STRONG	14529615;168

# PythonAstREPLTool con varios dfs

In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers.openai_tools import JsonOutputKeyToolsParser
from operator import itemgetter

from langchain_core.messages import ToolMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough

In [17]:
df_1 = pd.read_csv("../data/PromoterSet.tsv", sep="\t")
df_2 = pd.read_csv("../data/TUSet.tsv", sep="\t")
tool = PythonAstREPLTool(locals={"df_1": df_1, "df_2": df_2})
llm_with_tool = llm.bind_tools(tools=[tool], tool_choice=tool.name)
df_template = """\`\`\`python
{df_name}.head().to_markdown()
>>> {df_head}
\`\`\`"""
df_context = "\n\n".join(
    df_template.format(df_head=_df.head().to_markdown(), df_name=df_name)
    for _df, df_name in [(df_1, "df_1"), (df_2, "df_2")]
)

system = f"""You have access to a number of pandas dataframes. \
Here is a sample of rows from each dataframe and the python code that was used to generate the sample:

{df_context}

Given a user question about the dataframes, write the Python code to answer it. \
Don't assume you have access to any libraries other than built-in Python ones and pandas. \
Make sure to refer only to the variables mentioned above."""
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{question}"), ])

parser = JsonOutputKeyToolsParser(key_name=tool.name, first_tool_only=True)

chain = prompt | llm_with_tool | parser | tool
chain.invoke(
    {
        "question": "cuales son los nombres de los promotores que esten regulados por el sigma factor sigma54?"
    }
)

323      yahEp
324      yaaUp
325      crlp2
326     nikAp2
327      emrDp
         ...  
4002      nacp
4010     glnKp
4019     hyfAp
4026     zraPp
4027     zraSp
Name: pmName, Length: 101, dtype: object

In [18]:
df_1 = pd.read_csv("../data/PromoterSet.tsv", sep="\t")
df_2 = pd.read_csv("../data/TUSet.tsv", sep="\t")
tool = PythonAstREPLTool(locals={"df_1": df_1, "df_2": df_2})
llm_with_tool = llm.bind_tools(tools=[tool], tool_choice=tool.name)
df_template = """\`\`\`python
{df_name}.head().to_markdown()
>>> {df_head}
\`\`\`"""
df_context = "\n\n".join(
    df_template.format(df_head=_df.head().to_markdown(), df_name=df_name)
    for _df, df_name in [(df_1, "df_1"), (df_2, "df_2")]
)

system = f"""You have access to a number of pandas dataframes. \
Here is a sample of rows from each dataframe and the python code that was used to generate the sample:

{df_context}

Given a user question about the dataframes, write the Python code to answer it. \
Don't assume you have access to any libraries other than built-in Python ones and pandas. \
Make sure to refer only to the variables mentioned above."""
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{question}"), MessagesPlaceholder("chat_history", optional=True),])

def _get_chat_history(x: dict) -> list:
    """Parse the chain output up to this point into a list of chat history messages to insert in the prompt."""
    ai_msg = x["ai_msg"]
    tool_call_id = x["ai_msg"].additional_kwargs["tool_calls"][0]["id"]
    tool_msg = ToolMessage(tool_call_id=tool_call_id, content=str(x["tool_output"]))
    return [ai_msg, tool_msg]

parser = JsonOutputKeyToolsParser(key_name=tool.name, first_tool_only=True)

chain = (
    RunnablePassthrough.assign(ai_msg=prompt | llm_with_tool)
    .assign(tool_output=itemgetter("ai_msg") | parser | tool)
    .assign(chat_history=_get_chat_history)
    .assign(response=prompt | llm | StrOutputParser())
    .pick(["tool_output", "response"])
)

In [19]:
chain.invoke(
    {
        "question": "cuales son los nombres de los promotores que esten regulados por el sigma factor sigma54?"
    }
)

{'tool_output': 323      yahEp
 324      yaaUp
 325      crlp2
 326     nikAp2
 327      emrDp
          ...  
 4002      nacp
 4010     glnKp
 4019     hyfAp
 4026     zraPp
 4027     zraSp
 Name: pmName, Length: 101, dtype: object,
 'response': 'Los nombres de los promotores que están regulados por el sigma factor sigma54 son:\n\n- yahEp\n- yaaUp\n- crlp2\n- nikAp2\n- emrDp\n- nacp\n- glnKp\n- hyfAp\n- zraPp\n- zraSp\n\n(Estos son solo algunos ejemplos, hay un total de 101 promotores regulados por sigma54).'}